Mouse Click

In [ ]:
import cv2

img = cv2.imread("C:/Users/anton/PM 2.5/COMPGRAPH/myLove.jpg")
img = cv2.resize(img, (400, 600))

points = []  

def draw_line(event, x, y, flags, param):
    global points, img

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(f"Clicked point: {x}, {y}")  # nunjkin coor

        if len(points) == 2:
            cv2.line(img, points[0], points[1], (0, 0, 255), 2)  # red line
            print(f"Start point: {points[0]}, End point: {points[1]}")
            points = []  

cv2.namedWindow("My Love")
cv2.setMouseCallback("Draw Line", draw_line)
    
while True:
    cv2.imshow("Draw Line", img)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()


BruteForce

In [1]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def draw_line_bruteforce(x1, y1, x2, y2, color=(0,0,0)):
    if x1 == x2:  # garis vertikal nye
        for y in range(min(y1, y2), max(y1, y2)+1):
            if 0 <= x1 < width and 0 <= y < height:
                canvas[y, x1] = color
        
    elif y1 == y2:  # garis horizontal nye
        for x in range(min(x1, x2), max(x1, x2)+1):
            if 0 <= x < width and 0 <= y1 < height:
                canvas[y1, x] = color
    
    else:  # garis miring nye nih
        dx = x2 - x1
        dy = y2 - y1
        m = dy / dx
        for x in range(min(x1, x2), max(x1, x2)+1):
            y = int(m * (x - x1) + y1)
            if 0 <= x < width and 0 <= y < height:
                canvas[y, x] = color

# Markicob:
draw_line_bruteforce(50, 100, 50, 400, (255,0,0))   
draw_line_bruteforce(100, 400, 50, 400, (0,255,0))  
draw_line_bruteforce(100, 100, 400, 300, (0,0,255)) 

cv2.imshow("Brute Force Line Drawing", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


BruteForce Anti_Jagged (rounded edge)

In [7]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def blend_pixel(img, x, y, color, alpha):
    """Blend a pixel with transparency alpha (0..1)."""
    if 0 <= x < img.shape[1] and 0 <= y < img.shape[0]:
        img[y, x] = (1 - alpha) * img[y, x] + alpha * np.array(color)

def draw_line_antialiased(x1, y1, x2, y2, color=(0,0,0)):
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    steep = dy > dx

    if steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
        dx, dy = dy, dx

    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    gradient = dy / dx if dx != 0 else 0
    y = y1
    for x in range(x1, x2 + 1):
        if steep:
            blend_pixel(canvas, int(y), x, color, 1 - (y - int(y)))
            blend_pixel(canvas, int(y)+1, x, color, y - int(y))
        else:
            blend_pixel(canvas, x, int(y), color, 1 - (y - int(y)))
            blend_pixel(canvas, x, int(y)+1, color, y - int(y))
        y += gradient

# Contoh garis
draw_line_antialiased(50, 100, 400, 300, (1,1,250))

cv2.imshow("Anti-aliased Line (Wu's Algorithm)", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


Bresenham Algorithm

In [5]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def draw_line_bresenham(x1, y1, x2, y2, color=(0,0,0)):
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy

    while True:
        if 0 <= x1 < width and 0 <= y1 < height:
            canvas[y1, x1] = color
        if x1 == x2 and y1 == y2:
            break
        e2 = 2*err
        if e2 > -dy:
            err -= dy
            x1 += sx
        if e2 < dx:
            err += dx
            y1 += sy

# Contoh
draw_line_bresenham(50, 100, 400, 300, (255,0,0))   # miring
draw_line_bresenham(100, 200, 400, 200, (0,255,0))  # horizontal
draw_line_bresenham(300, 50, 300, 400, (0,0,255))   # vertikal

cv2.imshow("Bresenham Line Drawing", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


DDA (Digital Differential Analyzer)

In [2]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def draw_line_DDA(x1, y1, x2, y2, color=(0,0,0)):
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
    
    if steps == 0:  # jika titik sama
        canvas[y1, x1] = color
        return

    x_inc = dx / steps
    y_inc = dy / steps

    x, y = x1, y1
    for _ in range(steps+1):
        if 0 <= int(x) < width and 0 <= int(y) < height:
            canvas[int(round(y)), int(round(x))] = color
        x += x_inc
        y += y_inc

# Mari kita coba
draw_line_DDA(50, 100, 400, 300, (255,0,0))   # miring
draw_line_DDA(100, 200, 400, 200, (0,255,0))  # horizontal
draw_line_DDA(300, 50, 300, 400, (0,0,255))   # vertikal

cv2.imshow("DDA Line Drawing", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


Bressenham Line

In [ ]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def draw_line_bresenham(x1, y1, x2, y2, color=(0,0,0)):
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy

    while True:
        if 0 <= x1 < width and 0 <= y1 < height:
            canvas[y1, x1] = color
        if x1 == x2 and y1 == y2:
            break
        e2 = 2*err
        if e2 > -dy:
            err -= dy
            x1 += sx
        if e2 < dx:
            err += dx
            y1 += sy

# Markicob
draw_line_bresenham(50, 100, 400, 300, (255,0,0))   # miring
draw_line_bresenham(100, 200, 400, 200, (0,255,0))  # horizontal
draw_line_bresenham(300, 50, 300, 400, (0,0,255))   # vertikal

cv2.imshow("Bresenham Line Drawing", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


Bressenham Circle

In [3]:
import cv2
import numpy as np

width, height = 500, 500
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255

def draw_circle_bresenham(xc, yc, r, color=(0,0,0)):
    x = 0
    y = r
    p = 3 - 2 * r

    while x <= y:
        # 8-way symmetry
        points = [
            (xc+x, yc+y), (xc-x, yc+y),
            (xc+x, yc-y), (xc-x, yc-y),
            (xc+y, yc+x), (xc-y, yc+x),
            (xc+y, yc-x), (xc-y, yc-x)
        ]
        for px, py in points:
            if 0 <= px < width and 0 <= py < height:
                canvas[py, px] = color

        if p < 0:
            p = p + 4*x + 6
        else:
            p = p + 4*(x-y) + 10
            y -= 1
        x += 1

# Contoh: lingkaran dengan pusat (250,250), radius 100
draw_circle_bresenham(250, 250, 200, (255,0,0))

cv2.imshow("Bresenham Circle Drawing", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()
